In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import time
from datetime import datetime
from typing import Optional, List, Tuple

# APIキーの設定
try:
    GOOGLE_API_KEY: str = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError:
    print("シークレット 'GOOGLE_API_KEY' が見つかりません。ColabのシークレットマネージャーにAPIキーを設定してください。")
    exit()
except Exception as e:
    print(f"APIキーの設定中にエラーが発生しました: {e}")
    exit()

def query_model(model_name: str, prompt: str) -> bool:
    """
    指定されたモデルにプロンプトを送信し、結果と統計情報を表示します。

    Args:
        model_name (str): 使用するGenerative AIモデルの名前
        prompt (str): モデルに送信するプロンプトテキスト

    Returns:
        bool: モデルの実行が成功した場合True、失敗した場合False

    Raises:
        Exception: モデルの生成処理中にエラーが発生した場合
    """
    print(f"\n{'='*60}")
    print(f"LLMモデル: {model_name}")
    print(f"{'='*60}")

    start_time: float
    generation_time: float

    try:
        model = genai.GenerativeModel(model_name)

        # 生成開始時刻を記録
        start_time = time.time()
        start_datetime: datetime = datetime.now()

        print(f"生成開始時刻: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
        print("生成中...")

        # コンテンツ生成
        response = model.generate_content(prompt)

        # 生成終了時刻を記録
        end_time: float = time.time()
        end_datetime: datetime = datetime.now()
        generation_time = end_time - start_time

        # 結果表示
        print(f"\n生成結果:")
        print("-" * 40)
        print(response.text)
        print("-" * 40)

        # 統計情報表示
        print(f"\n生成統計:")
        print(f"   生成時間: {generation_time:.2f} 秒")
        print(f"   完了時刻: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

        # レスポンスオブジェクトから追加情報を取得（利用可能な場合）
        if hasattr(response, 'usage_metadata') and response.usage_metadata:
            usage = response.usage_metadata
            prompt_tokens: Optional[int] = getattr(usage, 'prompt_token_count', None)
            candidates_tokens: Optional[int] = getattr(usage, 'candidates_token_count', None)
            total_tokens: Optional[int] = getattr(usage, 'total_token_count', None)

            print(f"   プロンプトトークン数: {prompt_tokens if prompt_tokens is not None else 'N/A'}")
            print(f"   生成トークン数: {candidates_tokens if candidates_tokens is not None else 'N/A'}")
            print(f"   総トークン数: {total_tokens if total_tokens is not None else 'N/A'}")

        # 文字数統計
        response_length: int = len(response.text)
        words_count: int = len(response.text.split())
        print(f"   生成文字数: {response_length} 文字")
        print(f"   生成単語数: {words_count} 語")

        # 生成速度計算
        if generation_time > 0:
            chars_per_second: float = response_length / generation_time
            print(f"   生成速度: {chars_per_second:.1f} 文字/秒")

        print(f"   ステータス: 成功")
        return True

    except Exception as e:
        end_time = time.time()
        generation_time = end_time - start_time if 'start_time' in locals() else 0

        print(f"\nエラーが発生しました:")
        print(f"   エラー内容: {e}")
        print(f"   エラーまでの時間: {generation_time:.2f} 秒")
        print(f"   ステータス: 失敗")
        return False

    finally:
        print(f"{'='*60}\n")

def main() -> None:
    """
    複数のモデルでテストを実行し、全体の統計結果を表示します。

    設定されたモデルリストに対して順次テストを実行し、
    各モデルの結果と全体のサマリー統計を出力します。

    Returns:
        None

    Raises:
        Exception: APIキーの設定やモデルの実行中にエラーが発生した場合
    """
    models_to_test: List[str] = [
        "gemini-2.5-flash-preview-05-20",
        "gemma-3n-e4b-it"
    ]

    prompt_text: str = "真空中でもなぜ熱は伝わるのでしょうか？簡潔に説明してください。"

    print(f"Google Generative AI モデル比較テスト")
    print(f"プロンプト: \"{prompt_text}\"")
    print(f"テスト対象モデル数: {len(models_to_test)}")
    print(f"開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    overall_start_time: float = time.time()
    successful_models: int = 0
    failed_models: int = 0

    for i, model_id in enumerate(models_to_test, 1):
        print(f"\n{i}/{len(models_to_test)} 番目のモデルをテスト中...")
        try:
            success: bool = query_model(model_id, prompt_text)
            if success:
                successful_models += 1
            else:
                failed_models += 1
        except Exception as e:
            print(f"モデル {model_id} でエラー: {e}")
            failed_models += 1

    # 全体の統計表示
    overall_end_time: float = time.time()
    total_time: float = overall_end_time - overall_start_time
    average_time: float = total_time / len(models_to_test)

    print(f"{'='*60}")
    print(f"全体テスト結果")
    print(f"{'='*60}")
    print(f"成功: {successful_models} モデル")
    print(f"失敗: {failed_models} モデル")
    print(f"総実行時間: {total_time:.2f} 秒")
    print(f"平均実行時間: {average_time:.2f} 秒/モデル")
    print(f"完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*60}")


def setup_api_key() -> Optional[str]:
    """
    Google Colab のシークレットマネージャーからAPIキーを取得し、設定します。

    Returns:
        Optional[str]: 取得に成功した場合はAPIキー、失敗した場合はNone

    Raises:
        SystemExit: APIキーの取得または設定に失敗した場合
    """
    try:
        api_key: str = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=api_key)
        return api_key
    except userdata.SecretNotFoundError:
        print("シークレット 'GOOGLE_API_KEY' が見つかりません。ColabのシークレットマネージャーにAPIキーを設定してください")
        exit()
    except Exception as e:
        print(f"APIキーの設定中にエラーが発生しました: {e}")
        exit()


if __name__ == "__main__":
    GOOGLE_API_KEY: Optional[str] = setup_api_key()

    if GOOGLE_API_KEY is not None:
        main()
    else:
        print("APIキーが見つからなかったため、実行されませんでした")

Google Generative AI モデル比較テスト
プロンプト: "真空中でもなぜ熱は伝わるのでしょうか？簡潔に説明してください。"
テスト対象モデル数: 2
開始時刻: 2025-05-23 04:38:13

1/2 番目のモデルをテスト中...

LLMモデル: gemini-2.5-flash-preview-05-20
生成開始時刻: 2025-05-23 04:38:13
生成中...

生成結果:
----------------------------------------
真空中でも熱が伝わるのは、**熱放射（ねつほうしゃ）**によるものです。

*   熱の伝わり方には「伝導」「対流」「放射」の3種類がありますが、真空中には物質がないため、「伝導」や「対流」は起こりません。
*   しかし、**熱放射（赤外線などの電磁波）は物質を介さずにエネルギーを直接空間に放出します。**この電磁波が真空中を伝わり、別の物体に吸収されることで熱として伝わるのです。

太陽の熱が地球に届くのも、この熱放射によるものです。
----------------------------------------

生成統計:
   生成時間: 6.11 秒
   完了時刻: 2025-05-23 04:38:19
   プロンプトトークン数: 17
   生成トークン数: 135
   総トークン数: 1005
   生成文字数: 221 文字
   生成単語数: 6 語
   生成速度: 36.2 文字/秒
   ステータス: 成功


2/2 番目のモデルをテスト中...

LLMモデル: gemma-3n-e4b-it
生成開始時刻: 2025-05-23 04:38:19
生成中...

生成結果:
----------------------------------------
真空中でも熱が伝わるのは、**熱が物質の振動によって生じるエネルギーであり、その振動が波として伝わる**ためです。

*   **物質（固体、液体、気体）**：分子や原子が振動している。
*   **真空**：物質がないが、振動はエネルギーとして伝わる。
*   **伝播**：振動が近傍の分子を振動させ、その振動が連鎖的に伝わる。これが熱の伝わり方。

熱伝導の仕組みは、物質の